In [2]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

corpus = api.load('text8')

model = Word2Vec(corpus, vector_size=127, window=5, min_count=5, workers=4)



In [2]:
from src.dataset.data_loader import DataLoader
from src.dataset.data_reader import DatasetReader
data_reader = DatasetReader()
data_loader = DataLoader(eeg=data_reader.eeg, audio=data_reader.audio, stimulus=data_reader.stimulus)

╭──────────────────────────────────╮
│ 🚀 Initializing DatasetReader... │
╰──────────────────────────────────╯

🧑‍⚕️ Loading Data for Subject: 01

📂    Looking for file: sub-01_task-wordProduction

╭────────────────────────╮
│ 📡 Reading NWB Data... │
╰────────────────────────╯

✅ Data successfully loaded!

...

📝 Stimulus Data Shape: (100, 5)

📂  Preprocessing Audio...

🔄  Downsampling audio...

🎚️  Normalizing audio...

✅  Audio preprocessing complete!

In [3]:
words = data_loader.word_labels
words

['helft',
 'hierop',
 'stiekem',
 'en',
 'zevenduizend',
 'braadde',
 'vloog',
 'werd',
 'wegpakte',
 'wak',
 'zich',
 '9',
 '10',
 'tuwiet',
 'bakker',
 'hem',
 'kin',
 'stilstaan',
 'bevrijd',
 'aan',
 'wel',
 'was',
 'hoe',
 '8',
 'groen',
 'maantje',
 'vogelkooitje',
 '6',
 'teruggekregen',
 'wanneer',
 'juist',
 'zanddak',
 'lij',
 'of',
 'over',
 'dat',
 'betovering',
 'gefluit',
 '3',
 'zei',
 'bak',
 '5',
 'niet',
 'nachtegalen',
 'te',
 'komt',
 'van',
 'dauwdruppel',
 'met',
 'sterkste',
 'totdat',
 'helemaal',
 'mooie',
 'sprong',
 'ze',
 'smeekte',
 'pak',
 'boomstammen',
 'dan',
 'er',
 'onmiddellijk',
 'vogeltje',
 'dit',
 'door',
 'mooi',
 'een',
 'zijn',
 'nog',
 'op',
 'verstijfde',
 'al',
 'het',
 'haar',
 'redetwisten',
 'om',
 'kasteel',
 'vak',
 '7',
 'donkere',
 'struik',
 'naar',
 'maar',
 'ook',
 'verdwaald',
 '12',
 'dakker',
 'je',
 'direct',
 '2',
 'spreuk',
 'binnenplaats',
 'hij',
 'onschuldig',
 'als',
 '`s',
 'tussen',
 'zo',
 'meisjes',
 'veel',
 'schold

In [7]:
for word in words:
    embedding = model.wv[word]

    # Print the embedding
    print(f"Embedding for '{word}':{embedding.shape}")


KeyError: "Key 'helft' not present"

In [ ]:


# Load Dutch FastText embeddings
fasttext.util.download_model('nl', if_exists='ignore')  # Downloads Dutch model
ft = fasttext.load_model('cc.nl.300.bin')


 (100.00%) [==================================================>]                                                 ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]>                                                  ]> 

In [4]:
for word in words:
    
    # Get embedding
    embedding = ft.get_word_vector(word)
    print(f"Embedding for '{word}':", embedding.shape)


Embedding for 'helft': (300,)
Embedding for 'hierop': (300,)
Embedding for 'stiekem': (300,)
Embedding for 'en': (300,)
Embedding for 'zevenduizend': (300,)
Embedding for 'braadde': (300,)
Embedding for 'vloog': (300,)
Embedding for 'werd': (300,)
Embedding for 'wegpakte': (300,)
Embedding for 'wak': (300,)
Embedding for 'zich': (300,)
Embedding for '9': (300,)
Embedding for '10': (300,)
Embedding for 'tuwiet': (300,)
Embedding for 'bakker': (300,)
Embedding for 'hem': (300,)
Embedding for 'kin': (300,)
Embedding for 'stilstaan': (300,)
Embedding for 'bevrijd': (300,)
Embedding for 'aan': (300,)
Embedding for 'wel': (300,)
Embedding for 'was': (300,)
Embedding for 'hoe': (300,)
Embedding for '8': (300,)
Embedding for 'groen': (300,)
Embedding for 'maantje': (300,)
Embedding for 'vogelkooitje': (300,)
Embedding for '6': (300,)
Embedding for 'teruggekregen': (300,)
Embedding for 'wanneer': (300,)
Embedding for 'juist': (300,)
Embedding for 'zanddak': (300,)
Embedding for 'lij': (300,)
Em

In [5]:
embedding

array([-1.54359071e-02, -5.91878640e-03, -7.14253774e-03, -7.75813982e-02,
       -7.33652562e-02,  1.09428307e-02,  4.13897401e-03, -2.87661832e-02,
       -4.75232909e-03, -3.27637754e-02,  7.06499740e-02, -7.98742026e-02,
        1.33666575e-01, -1.90426167e-02,  1.69411786e-02, -6.61991090e-02,
       -2.39026006e-02,  4.26105373e-02,  8.12551603e-02, -2.02665548e-03,
       -8.56545344e-02, -1.21716596e-02,  9.09481198e-02, -1.72472615e-02,
       -1.09126367e-01, -3.48004885e-02, -6.87091947e-02, -1.06344745e-02,
        2.53844839e-02,  1.49284946e-02,  4.14293706e-02, -4.66428772e-02,
       -8.61971304e-02, -8.00794289e-02, -3.96828167e-02, -5.14151156e-02,
       -2.23702546e-02, -1.34129380e-03, -4.03983146e-02,  3.56481001e-02,
       -1.01458706e-01, -4.28304076e-02, -2.35032849e-02,  5.33079989e-02,
       -8.18240419e-02,  2.91029047e-02,  2.09411625e-02,  8.58800337e-02,
       -2.63204016e-02, -4.49776091e-02,  5.74861169e-02, -6.40638499e-03,
        3.65345627e-02, -